# Use Automated Machine Learning

There are many kinds of machine learning algorithm that you can use to train a model, and sometimes it's not easy to determine the most effective algorithm for your particular data and prediction requirements. Additionally, you can significantly affect the predictive performance of a model by preprocessing the training data, using techniques such as normalization, missing feature imputation, and others. In your quest to find the *best* model for your requirements, you may need to try many combinations of algorithms and preprocessing transformations; which takes a lot of time and compute resources.

Azure Machine Learning enables you to automate the comparison of models trained using different algorithms and preprocessing options. You can use the visual interface in [Azure Machine Learning studio](https://ml/azure.com) or the SDK to leverage this capability. The SDK gives you greater control over the settings for the automated machine learning experiment, but the visual interface is easier to use.

## Before you start

In addition to the latest version of the **azureml-sdk** and **azureml-widgets** packages, you'll need the **azureml-train-automl** package to run the code in this notebook. Run the cell below to verify that it is installed.

In [1]:
pip show azureml-train-automl

Name: azureml-train-automl
Version: 1.41.0
Summary: Used for automatically finding the best machine learning model and its parameters.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: azureml-dataset-runtime, azureml-train-automl-client, azureml-automl-core, azureml-automl-runtime, azureml-train-automl-runtime
Required-by: azureml-automl-dnn-nlp
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.41.0 to work with nahmed30-azureml-workspace


## Prepare data

You don't need to create a training script for automated machine learning, but you do need to load the training data. In this case, you'll use a dataset containing details of email ham and spam, and then split this into two datasets: one for training, and another for model validation.

In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name

from azureml.core import Dataset


found = False
key = "UdacityPrjEmailSpamDataSet"
description_text = "Email Spam Detection DataSet for Udacity Capstone Proj "

dataset = None
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        spam_data = 'https://www.kaggle.com/code/rumbleftw/beginner-friendly-spam-ham-sms-classification/data#:~:text=calendar_view_week-,spam,-.csv'
        dataset = Dataset.Tabular.from_delimited_files(spam_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()



,v1,v2,Column3,Column4,Column5
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [4]:
# Split the dataset into training and validation subsets

# emailspam_ds = ws.datasets.get("email spam dataset")

train_ds, test_ds = dataset.random_split(percentage=0.8, seed=40)
print("Data ready!")

Data ready!


## Prepare compute

One of the benefits of cloud compute is that it scales on-demand, enabling you to provision enough compute resources to process multiple child-runs of an automated machine learning experiment in parallel.

Use the following code to specify an Azure Machine Learning compute cluster (it will be created if it doesn't already exist)..

> **Important**: Change *your-compute-cluster* to the name of your compute cluster in the code below before running it! Cluster names must be globally unique names between 2 to 16 characters in length. Valid characters are letters, digits, and the - character.

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

Found existing cluster, use it.


> **Note**: Compute instances and clusters are based on standard Azure virtual machine images. For this exercise, the *Standard_DS11_v2* image is recommended to achieve the optimal balance of cost and performance. If your subscription has a quota that does not include this image, choose an alternative image; but bear in mind that a larger image may incur higher cost and a smaller image may not be sufficient to complete the tasks. Alternatively, ask your Azure administrator to extend your quota.

## Configure automated machine learning

Now you're ready to configure the automated machine learning experiment.

One of the most important configuration settings is the metric by which model performance should be evaluated. You can retrieve a list of the metrics that are calculated by automated machine learning for a particular type of model task (classification or regression) like this:

In [6]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

accuracy
precision_score_weighted
average_precision_score_weighted
AUC_weighted
norm_macro_recall


Having decided the metric you want to optimize (in this example, *AUC_weighted*), you can configure the automated machine learning run. To do this, you'll need an AutoML configuration that specifies the target metric as well as options like the data to use, how many combinations to try, and so on.

> **Note**: In this example, you'll restrict the experiment to 4 iterations to reduce the amount of time taken. In reality, you'd likely try many more iterations.

In [7]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='v1',
                             iterations=4,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


## Run an automated machine learning experiment

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [8]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'mslearn-emailspam-automl-sdk')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
mslearn-emailspam-automl-sdk,AutoML_99072b9e-0c02-4396-a0a8-fd3a4258e5d0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
mslearn-emailspam-automl-sdk,AutoML_99072b9e-0c02-4396-a0a8-fd3a4258e5d0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in th

{'runId': 'AutoML_99072b9e-0c02-4396-a0a8-fd3a4258e5d0',
 'target': 'your-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-01T02:54:50.448985Z',
 'endTimeUtc': '2022-09-01T02:58:38.648815Z',
 'services': {},
 'properties': {'num_iterations': '4',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'your-compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment","subscription_id":"16bc73b5-82be-47f2-b5ab-f2373344794c","resource_group":"epe-poc-nazeer","workspace_name":"nahmed30-azureml-workspace","region":"centralus","compute_target":"your-compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"i

## View child run details

When the experiment has completed, view the output in the widget, and click the run that produced the best result to see its details.

Then click the link to view the experiment details in Azure Machine Learning studio and view the overall experiment details before viewing the details for the individual run that produced the best result. There's lots of information here about the performance of the model generated.

You can also retrieve all of the child runs and view their metrics using the SDK:

In [8]:
for run in automl_run.get_children():
    print('Run ID', run.id)
    for metric in run.get_metrics():
        print('\t', run.get_metrics(metric))

Run ID AutoML_d18f6632-3aad-4e20-9fc3-7db8e1b6ee85_3
	 {'balanced_accuracy': 0.9524148348466188}
	 {'precision_score_macro': 0.9593709366408918}
	 {'f1_score_weighted': 0.9602187289582469}
	 {'AUC_weighted': 0.9921314599553309}
	 {'average_precision_score_micro': 0.990556481789572}
	 {'precision_score_micro': 0.9603698811096433}
	 {'precision_score_weighted': 0.9603066234124137}
	 {'AUC_micro': 0.9905042054770961}
	 {'average_precision_score_weighted': 0.9923057515131282}
	 {'f1_score_macro': 0.9557471444794352}
	 {'f1_score_micro': 0.9603698811096433}
	 {'accuracy': 0.9603698811096433}
	 {'recall_score_macro': 0.9524148348466188}
	 {'matthews_correlation': 0.911759236714881}
	 {'log_loss': 0.12602103831393655}
	 {'weighted_accuracy': 0.9669083028807882}
	 {'average_precision_score_macro': 0.9906786790415391}
	 {'AUC_macro': 0.992131459955331}
	 {'recall_score_weighted': 0.9603698811096433}
	 {'recall_score_micro': 0.9603698811096433}
	 {'norm_macro_recall': 0.9048296696932376}
	 {'acc

## Get the best run

You can retrieve the best-performing run, and view its details like this:

In [9]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.44.0, current version:1.41.0
Package:azureml-core, training version:1.44.0, current version:1.41.0
Package:azureml-dataprep, training version:4.2.2, current version:3.1.1
Package:azureml-dataprep-rslex, training version:2.8.1, current version:2.5.2
Package:azureml-dataset-runtime, training version:1.44.0, current version:1.41.0
Package:azureml-defaults, training version:1.44.0, current version:1.41.0
Package:azureml-inference-server-http, training version:0.7.4, current version:0.4.13
Package:azureml-interpret, training version:1.44.0, current version:1.41.0
Package:azureml-mlflow, training version:1.44.0, current version:1.41.0
Package:azureml-pipeline-core, training version:1.44.0, current version:1.41.0
Package:azureml-responsibleai, training version:1.44.0, current version:1.41.0
Package:azureml-telemetry, training version:1.44.0, current version:1.41.0
Package:azureml-train-automl-client, training version:1.44.0, current version:1

Run(Experiment: mslearn-diabetes-automl-sdk,
Id: AutoML_d18f6632-3aad-4e20-9fc3-7db8e1b6ee85_2,
Type: azureml.scriptrun,
Status: Completed)

Best Model Definition:
Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/nahmed30-computeinstance/code/Users/nahmed30/mslearn-dp100')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('1', Pipeline(steps=..._info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), tree_method='auto'))])), ('0', Pipeline(steps=[('maxabsscaler', MaxAbsScaler()), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'})))]))], weights=[0.6666666666666666, 0.3333333333333333]))])

Best Run Transformations:
datatransformer
prefittedsoftvotingc

Finally, having found the best performing model, you can register it.

In [10]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='emailspam_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 7
	 Training context : Auto ML
	 AUC : 0.9921701418400241
	 Accuracy : 0.9619110523998239


AutoML51fd6e53449 version: 1


diabetes_model version: 6
	 Training context : Hyperdrive
	 AUC : 0.988579354468807
	 Accuracy : 0.9457777777777778


diabetes_model version: 5
	 Training context : Hyperdrive
	 AUC : 0.9885804604667666
	 Accuracy : 0.9457777777777778


diabetes_model version: 4
	 Training context : Hyperdrive
	 AUC : 0.9885804604667666
	 Accuracy : 0.9457777777777778


diabetes_model version: 3
	 Training context : Hyperdrive
	 AUC : 0.9885804604667666
	 Accuracy : 0.9457777777777778


diabetes_model version: 2
	 Training context : Hyperdrive
	 AUC : 0.9885804604667666
	 Accuracy : 0.9457777777777778


diabetes_model version: 1
	 Training context : Hyperdrive
	 AUC : 0.9885804604667666
	 Accuracy : 0.9457777777777778


AutoML9a459769542 version: 1


AutoML112d464ef52 version: 1


AutoML07bb052bd46 version: 1


AutoML75205f5c844 version: 1


AutoML41fdcaffc4

> **More Information**: For more information Automated Machine Learning, see the [Azure ML documentation](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train).